# COVID-19 TEST RESULTS DASHBOARD UPDATES
### <a href="https://hdma-sdsu.github.io/index.html">COVID-19 Research Hub</a>
Center for Human Dynamics in the Mobile Age (HDMA) at SDSU <br>
Jessica Embury

#### IMPORT STATEMENTS

In [118]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
import json
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import sys
import tabula
import webbrowser

#### VARIABLES: DATES, PATHS, AGOL IDS

In [119]:
######################
# USER ENTERED DATES #
######################
most_recent_date = '11/18/20'  # file folder date
data_date = '11/14/20'  # date through
prior_date = '11/07/20'  # date through of prior week
day_after_prior = '11/8/20'  # one date after the prior date

#########
# PATHS #
#########
# pdf file path
file = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/{}20/Summary_of_All_Tests_Reported_by_Zip_Code_of_Residence.pdf'.format(most_recent_date.replace('/', '-'))

# local folder for storing csv files
local_dir = 'C:/Users/jesse/HDMA/tests'

# csv files
new_csv = '{}/covid_tests_total_{}.csv'.format(local_dir, data_date.replace('/',''))  # this week's total results
change_csv = '{}/covid_tests_change_{}.csv'.format(local_dir, data_date.replace('/',''))  # this week's new results
prior_csv = '{}/covid_tests_total_{}.csv'.format(local_dir, prior_date.replace('/',''))  # last week's total results
zips_csv = '{}/zip_info.csv'.format(local_dir)  # zip code spatial data
pops_csv = '{}/pop_info.csv'.format(local_dir)  # population data
total_upload = '{}/sd_zips_covid_test_results.csv'.format(local_dir)  # csv for upload to AGOL
change_upload = '{}/sd_zips_covid_tests_new.csv'.format(local_dir)  # csv for upload to AGOL

############
# AGOL IDs #
############
total_layer_id = '8ae9d6bc96ff4ac7b6240696c77fc4b6'
weekly_layer_id = '812242e64749440baf7e1281533f464a'

total_map_id = 'f218f594126140ed939d2b5e7baa8baa'
weekly_map_id = 'ad5c0361faa44573adcb9c5b3a8dedb0'

total_dash = 'https://arcg.is/11nm9S'
weekly_dash = 'https://arcg.is/1qmieu'

# CUMULATIVE TEST RESULTS

#### USE TABULA TO CONVERT PDF TO TABLE

In [120]:
tables = tabula.read_pdf(file, pages = "all", multiple_tables = True)
tables[0]

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,NaN,NaN,NaN,Percent,Tests per,NaN,NaN,NaN,NaN,Percent,Tests per
1,Zip Code,Positive,Negative Indeterminate,Total,Postive,"100,000*",Zip Code,Positive Negative Indeterminate,Total,NaN,Positive,"100,000*"
2,91901,189,"6,395 10","6,594",2.9%,"36,662",92071,"808 27,550","61 28,419",NaN,2.8%,"48,819"
3,91902,425,"7,835 27","8,287",5.1%,"47,002",92075,"104 6,085","9 6,198",NaN,1.7%,"47,761"
4,91905,9,511 0,520,1.7%,*,92078,"701 17,104","40 17,845",NaN,3.9%,"35,009"
...,...,...,...,...,...,...,...,...,...,...,...,...
56,92065,657,"10,396 25","11,078",5.9%,"30,555",Subtotal,"69,789 1,400,787","3,921 1,474,497",NaN,4.7%,"43,991"
57,92066,4,61 0,65,6.2%,*,NaN,NaN,NaN,NaN,NaN,NaN
58,92067,98,"5,058 2","5,158",1.9%,*,**Other,"3,664 67,754","128 71,546",NaN,5.1%,*
59,92069,995,"14,579 74","15,648",6.4%,"30,580",***Unknown,"3,287 91,818","204 95,309",NaN,3.4%,*


#### CREATE TABLE WITH LEFT COLUMN ZIPS, CLEAN DATA

In [121]:
# rename columns
t1 = tables[0][[0, 1, 2, 3, 4, 5]]
t1 = t1.rename(columns = {0: 'Zip', 1: 'Positive', 2: 'Neg Indet', 3: 'Total', 
                         4: 'Percent Positive', 5: 'Tests Per 100K'})

# drop bad rows
t1 = t1.drop(t1.index[0:2])

# create new columns for clean data
t1['Negative'] = 'NaN'
t1['Indeterminate'] = 'NaN'

# clean data
for i, row in t1.iterrows():
    temp = t1['Neg Indet'][i]
    temp_list = temp.split( )
    t1['Negative'][i] = temp_list[0]
    t1['Indeterminate'][i] = temp_list[1]
    if len(temp_list) == 3:
        t1['Total'][i] = temp_list[2]

# delete bad columns
del t1['Neg Indet']

print(len(t1))
t1.head()

59


,Zip,Positive,Total,Percent Positive,Tests Per 100K,Negative,Indeterminate
2,91901,189,"6,594",2.9%,"36,662","6,395",10
3,91902,425,"8,287",5.1%,"47,002","7,835",27
4,91905,9,520,1.7%,*,511,0
5,91906,65,"1,639",4.0%,*,"1,570",4
6,91910,"2,979","44,562",6.7%,"53,155","41,460",123


#### CREATE TABLE WITH RIGHT COLUMN ZIPS, CLEAN DATA

In [122]:
# rename columns
t2 = tables[0][[6, 7, 8, 9, 10, 11]]
t2 = t2.rename(columns = {6: 'Zip', 7: 'Pos Neg', 8: 'Indet Total', 9: 'Total', 
                         10: 'Percent Positive', 11: 'Tests Per 100K'})

# delete bad rows
t2 = t2.drop(t2.index[0:2])
t2 = t2.drop(t2.index[55])

# create new columns for clean data
t2['Negative'] = 'NaN'
t2['Indeterminate'] = 'NaN'

# clean data
for i, row in t2.iterrows():
    temp = t2['Indet Total'][i]
    temp_list = str(temp).split( )
    t2['Indeterminate'][i] = temp_list[0]
    t2['Total'][i] = temp_list[1]
    
    temp2 = t2['Pos Neg'][i]
    temp_list2 = temp2.split( )
    t2['Pos Neg'][i] = temp_list2[0]
    t2['Negative'][i] = temp_list2[1]

t2 = t2.rename(columns = {'Pos Neg':'Positive'})

# delete bad columns
del t2['Indet Total']

print(len(t2))
t2.head()

58


,Zip,Positive,Total,Percent Positive,Tests Per 100K,Negative,Indeterminate
2,92071,808,"28,419",2.8%,"48,819","27,550",61
3,92075,104,"6,198",1.7%,"47,761","6,085",9
4,92078,701,"17,845",3.9%,"35,009","17,104",40
5,92081,363,"10,117",3.6%,"30,759","9,710",44
6,92082,249,"6,059",4.1%,"34,272","5,794",16


#### MERGE INTO ONE DATAFRAME

In [123]:
# merge t1 and t2
t1 = t1.append(t2, ignore_index=True)

# new dataframe with properly ordered columns
df = t1[['Zip', 'Positive', 'Negative', 'Indeterminate', 'Total', 'Percent Positive', 'Tests Per 100K']]
df = df.rename(columns = {'ZIP':'Zip'})

# format data
for i,row in df.iterrows():
    df['Positive'][i] = int(df['Positive'][i].replace(',',''))
    df['Negative'][i] = int(df['Negative'][i].replace(',',''))
    df['Indeterminate'][i] = int(df['Indeterminate'][i].replace(',',''))   
    df['Total'][i] = int(df['Total'][i].replace(',',''))
    df['Percent Positive'][i] = round((df['Positive'][i]/df['Total'][i]*100), 1)

# add date column
df['Date'] = data_date

print(len(df))
df.head()

117


,Zip,Positive,Negative,Indeterminate,Total,Percent Positive,Tests Per 100K,Date
0,91901,189,6395,10,6594,2.9,"36,662",11/14/20
1,91902,425,7835,27,8287,5.1,"47,002",11/14/20
2,91905,9,511,0,520,1.7,*,11/14/20
3,91906,65,1570,4,1639,4,*,11/14/20
4,91910,2979,41460,123,44562,6.7,"53,155",11/14/20


#### ADD COMMUNITY NAMES AND COORDINATES

In [124]:
# dataframe with zip code spatial data
zips = pd.read_csv(zips_csv)
zips['Zip'] = zips['Zip'].astype(str)

# merge test data with zip code data
zips = zips.merge(df, on='Zip')
zips.head()

,Zip,Community,Latitude,Longitude,Positive,Negative,Indeterminate,Total,Percent Positive,Tests Per 100K,Date
0,91901,Alpine,32.805710,-116.695537,189,6395,10,6594,2.9,"36,662",11/14/20
1,91902,Bonita,32.671583,-117.015068,425,7835,27,8287,5.1,"47,002",11/14/20
2,91905,Boulevard,32.718365,-116.305469,9,511,0,520,1.7,*,11/14/20
3,91906,Campo,32.660427,-116.469681,65,1570,4,1639,4,*,11/14/20
4,91910,Chula Vista,32.636413,-117.065653,2979,41460,123,44562,6.7,"53,155",11/14/20


#### ADD POPULATION DATA AND CALCULATE POSITIVE/100K

In [125]:
# create and format dataframe with population data
pops = pd.read_csv(pops_csv)
pops['Zip'] = pops['Zip'].astype(str)
del pops['Community']

# merge test result data and population data
zips = zips.merge(pops, on='Zip')

# calculate positive per 100k
zips['Positive Cases Per 100K'] = ''
for i, row in zips.iterrows():
    if zips['Population 2018'][i] >= 1000:
        zips['Positive Cases Per 100K'][i] = round(zips['Positive'][i]/zips['Population 2018'][i]*100000, 0)
    else:
        zips['Positive Cases Per 100K'][i] = '*'

# save as csv file
zips.to_csv(new_csv, index=False)
zips.to_csv(total_upload, index=False)  # for AGOL upload
zips.head()

,Zip,Community,Latitude,Longitude,Positive,Negative,Indeterminate,Total,Percent Positive,Tests Per 100K,Date,Population 2018,Positive Cases Per 100K
0,91901,Alpine,32.805710,-116.695537,189,6395,10,6594,2.9,"36,662",11/14/20,17885,1057
1,91902,Bonita,32.671583,-117.015068,425,7835,27,8287,5.1,"47,002",11/14/20,17375,2446
2,91905,Boulevard,32.718365,-116.305469,9,511,0,520,1.7,*,11/14/20,2014,447
3,91906,Campo,32.660427,-116.469681,65,1570,4,1639,4,*,11/14/20,3686,1763
4,91910,Chula Vista,32.636413,-117.065653,2979,41460,123,44562,6.7,"53,155",11/14/20,82682,3603


# WEEKLY TEST RESULTS

#### COMBINE CUMULATIVE TEST DATA (MOST RECENT WEEK AND PRIOR WEEK)

In [126]:
# create dataframe with culumative results from prior week
prior = pd.read_csv(prior_csv)

prior = prior.rename(columns = {'Positive':'old pos', 'Negative':'old neg', 'Indeterminate':'old indet', 
                                'Total':'old total', 'Date':'old date'    
})

# create dataframe with cumulative results from this week
new = pd.read_csv(new_csv)
new = new.rename(columns = {'Positive':'new pos', 'Negative':'new neg', 'Indeterminate':'new indet', 
                                'Total':'new total', 'Date':'new date'    
})

# combine into one dataframe for weekly values
change = prior[['Zip','Community','Latitude','Longitude','old pos','old neg', 'old indet', 'old total', 'old date']].merge(
    new[['Zip', 'new pos', 'new neg', 'new indet', 'new total', 'new date']], on='Zip')

change.head()

,Zip,Community,Latitude,Longitude,old pos,old neg,old indet,old total,old date,new pos,new neg,new indet,new total,new date
0,91901,Alpine,32.805710,-116.695537,170,6088,10,6268,11/7/20,189,6395,10,6594,11/14/20
1,91902,Bonita,32.671583,-117.015068,399,7336,18,7753,11/7/20,425,7835,27,8287,11/14/20
2,91905,Boulevard,32.718365,-116.305469,9,488,0,497,11/7/20,9,511,0,520,11/14/20
3,91906,Campo,32.660427,-116.469681,58,1472,3,1533,11/7/20,65,1570,4,1639,11/14/20
4,91910,Chula Vista,32.636413,-117.065653,2820,39189,97,42106,11/7/20,2979,41460,123,44562,11/14/20


#### CALCULATE NEW WEEKLY VALUES

In [127]:
# add columns for weekly values
change['Positive'] = 0
change['Negative'] = 0
change['Indeterminate'] = 0
change['Total'] = 0
change['Percent Positive'] = 0
change['Date Range'] = '{} - {}'.format(day_after_prior, data_date)

# calculate weekly values
for i, row in change.iterrows():
    change['Positive'][i] = change['new pos'][i] - change['old pos'][i]
    change['Negative'][i] = change['new neg'][i] - change['old neg'][i]
    change['Indeterminate'][i] = change['new indet'][i] - change['old indet'][i]
    change['Total'][i] = change['new total'][i] - change['old total'][i]
    change['Percent Positive'] = round(change['Positive']/change['Total']*100, 1)

# delete old/new cumulative values
del change['old pos']
del change['old neg']
del change['old indet']
del change['old total']
del change['old date']
del change['new pos']
del change['new neg']
del change['new indet']
del change['new total']
del change['new date']

change.head()

,Zip,Community,Latitude,Longitude,Positive,Negative,Indeterminate,Total,Percent Positive,Date Range
0,91901,Alpine,32.805710,-116.695537,19,307,0,326,5.8,11/8/20 - 11/14/20
1,91902,Bonita,32.671583,-117.015068,26,499,9,534,4.9,11/8/20 - 11/14/20
2,91905,Boulevard,32.718365,-116.305469,0,23,0,23,0.0,11/8/20 - 11/14/20
3,91906,Campo,32.660427,-116.469681,7,98,1,106,6.6,11/8/20 - 11/14/20
4,91910,Chula Vista,32.636413,-117.065653,159,2271,26,2456,6.5,11/8/20 - 11/14/20


#### ADD POPULATION DATA AND CALCULATE POSITIVE/100K

In [128]:
# add population data
change['Zip'] = change['Zip'].astype(str)
change = change.merge(pops, on='Zip')

# calculate positive per 100k
change['Positive Cases Per 100K'] = ''
for i, row in change.iterrows():
    if change['Population 2018'][i] >= 1000:
        change['Positive Cases Per 100K'][i] = round(change['Positive'][i]/change['Population 2018'][i]*100000, 0)
    else:
        change['Positive Cases Per 100K'][i] = '*'

# save as csv
change.to_csv(change_csv, index=False)
change.to_csv(change_upload, index=False)  # for AGOL upload
change.head()

,Zip,Community,Latitude,Longitude,Positive,Negative,Indeterminate,Total,Percent Positive,Date Range,Population 2018,Positive Cases Per 100K
0,91901,Alpine,32.805710,-116.695537,19,307,0,326,5.8,11/8/20 - 11/14/20,17885,106
1,91902,Bonita,32.671583,-117.015068,26,499,9,534,4.9,11/8/20 - 11/14/20,17375,150
2,91905,Boulevard,32.718365,-116.305469,0,23,0,23,0.0,11/8/20 - 11/14/20,2014,0
3,91906,Campo,32.660427,-116.469681,7,98,1,106,6.6,11/8/20 - 11/14/20,3686,190
4,91910,Chula Vista,32.636413,-117.065653,159,2271,26,2456,6.5,11/8/20 - 11/14/20,82682,192


# UPDATE AGOL DASHBOARDS

#### CONNECT TO AGOL ACCOUNT

In [129]:
gis = GIS("pro")

#### FUNCTIONS FOR MAP MODIFICATIONS

In [130]:
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''    
    m = gis.content.get(map_id)
    data = m.get_data()
    print(m)    
    return data

def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''
    m = gis.content.get(map_id)  
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}
    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)    
    return update

#### OVERWRITE FEATURE LAYERS

In [131]:
#########
# TOTAL #
#########

#get feature layer 
total_layer = gis.content.get(total_layer_id)
total_layer

total_layer_collection = FeatureLayerCollection.fromitem(total_layer)

#call the overwrite() method which can be accessed using the manager property
total_layer_collection.manager.overwrite(total_upload)

{'success': True}

In [132]:
##########
# WEEKLY #
##########

#get feature layer 
weekly_layer = gis.content.get(weekly_layer_id)
weekly_layer

weekly_layer_collection = FeatureLayerCollection.fromitem(weekly_layer)

#call the overwrite() method which can be accessed using the manager property
weekly_layer_collection.manager.overwrite(change_upload)

{'success': True}

#### UPDATE MAP SYMBOLOGY

In [133]:
#########
# TOTAL #
#########

#get max for graduated point symbology
total_max = zips['Total'].max()
total_max = numpy.int64(total_max)
print(total_max)

#get map data
total_map = get_map(total_map_id)

#set max value for graduated points symbols
total_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = total_max.item()
total_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = total_max.item()

#update map to save changes
total_map_update = update_map(total_map_id, total_map)
total_map_update

45186
<Item title:"San Diego Zip Code COVID-19 Testing: Total Tests and Percent Positive" type:Web Map owner:jembury8568_SDSUGeo>


True

In [134]:
##########
# WEEKLY #
##########

#get max for graduated point symbology
weekly_max = change['Total'].max()
print(weekly_max)

#get map data
weekly_map = get_map(weekly_map_id)

#set max value for graduated points symbols
weekly_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = weekly_max.item()
weekly_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = weekly_max.item()

#update map to save changes
weekly_map_update = update_map(weekly_map_id, weekly_map)
weekly_map_update

2900
<Item title:"San Diego Zip Code COVID-19 Testing: New Tests and Percent Positive" type:Web Map owner:jembury8568_SDSUGeo>


True

#### CHECK DASHBOARDS

In [135]:
webbrowser.open(total_dash, new=2)
webbrowser.open(weekly_dash, new=2)

True